# Get Economy Data

http://data.nsdi.go.kr/dataset/20200305ds00002

In [1]:
from datetime import datetime
from time import time
import pandas as pd
import numpy as np
from glob import glob
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
from tqdm.notebook import tqdm
from ko_pron import romanise
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import urllib
from urllib.request import urlopen
import json
import pickle
from urllib.parse import quote_plus, urlencode

# 1. 공시지가 데이터

## (1) 데이터 로드

In [2]:
economy_path = '../data/공시지가'
economy_path_list = sorted(glob(economy_path+'/*.txt'))
economy_path_list

['../data/공시지가\\APMM_NV_JIGA_MNG_11_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_26_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_27_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_28_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_29_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_30_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_31_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_36_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_41_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_42_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_43_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_44_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_45_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_46_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_47_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_48_2021_01.txt',
 '../data/공시지가\\APMM_NV_JIGA_MNG_50_2021_01.txt']

## (2) 샘플 데이터 출력

In [3]:
pd.read_csv(economy_path_list[0], sep='|', engine='python', encoding='cp949')

,PNU,BASE_YEAR,STDMT,PNILP,PJJI_YN,PANN_YMD,ETC_CNTN,COL_ADM_SECT_CD
0,1111010600100070038,2021,1,7460000,1,20210531,NaN,11110
1,1111010600100070040,2021,1,2008000,0,20210531,NaN,11110
2,1111010600100070045,2021,1,4535000,0,20210531,NaN,11110
3,1111010600100080000,2021,1,4916000,0,20210531,NaN,11110
4,1111010600100090001,2021,1,4820000,0,20210531,NaN,11110
...,...,...,...,...,...,...,...,...
909146,1174010800104370032,2021,1,4375000,0,20210531,NaN,11740
909147,1174010800104370033,2021,1,4375000,0,20210531,NaN,11740
909148,1174010800104370034,2021,1,1364000,0,20210531,NaN,11740
909149,1174010800104370036,2021,1,5748000,0,20210531,NaN,11740


## (3) 데이터 전처리

In [4]:
economy_data = {}
economy_df = pd.DataFrame()
for i in range(len(economy_path_list)):
    economy_data[i] = pd.read_csv(economy_path_list[i], sep='|', engine='python', encoding='cp949')
    economy_data[i].PNU = economy_data[i].PNU.str.slice(0, 4)
    economy_df = pd.concat([economy_df, pd.DataFrame(economy_data[i].groupby('PNU')['PNILP'].mean())])
economy_df = economy_df.reset_index().rename(columns={"PNILP":"price"})
economy_df.price = np.floor(economy_df.price/1000)
economy_df

,PNU,price
0,1111,5789.0
1,1114,9763.0
2,1117,6553.0
3,1120,4353.0
4,1121,4157.0
...,...,...
223,4887,26.0
224,4888,32.0
225,4889,20.0
226,5011,223.0


# 2. PNU좌표 변환을 위한 법정동코드 데이터

## (1) 데이터 로드

In [5]:
legal_code = pd.read_csv('../data/legal_code.csv', dtype="O")
legal_code

,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,NaN,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,NaN,19880423,NaN
2,1111010100,서울특별시,종로구,청운동,NaN,19880423,NaN
3,1111010200,서울특별시,종로구,신교동,NaN,19880423,NaN
4,1111010300,서울특별시,종로구,궁정동,NaN,19880423,NaN
...,...,...,...,...,...,...,...
20558,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,NaN
20559,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,NaN
20560,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,NaN
20561,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,NaN


## (2) 데이터 전처리

In [6]:
legal_code_1 = legal_code[['법정동코드', '시도명']].rename(columns={"법정동코드":"PNU", "시도명":"loc_name"})
legal_code_1.PNU = legal_code_1.PNU.str.slice(0, 4)
legal_code_1 = legal_code_1.drop_duplicates(subset='loc_name', keep='last').dropna().reset_index(drop=True)
for i in range(len(legal_code_1)):
    legal_code_1.loc_name[i] = romanise(legal_code_1.loc_name[i], 'rr').replace('-', '').title()
legal_code_1.loc_name = legal_code_1.loc_name.str.replace('gwangyeoksi', '')
legal_code_1.loc_name = legal_code_1.loc_name.str.replace('teukbyeoljachido', '')
legal_code_1.loc_name = legal_code_1.loc_name.str.replace('teukbyeolsi', '')
legal_code_1 = legal_code_1.drop_duplicates('loc_name')
legal_code_1

,PNU,loc_name
0,1174,Seoul
1,2671,Busan
2,2771,Daegu
3,2872,Incheon
4,2920,Gwangju
5,3023,Daejeon
6,3171,Ulsan
7,3611,Sejongteukbyeoljachisi
8,4110,Bukbuchuljangso
9,4183,Gyeonggido


In [7]:
legal_code_2 = legal_code[['법정동코드', '시군구명']].rename(columns={"법정동코드":"PNU", "시군구명":"loc_name"})
legal_code_2.PNU = legal_code_2.PNU.str.slice(0, 4)
legal_code_2 = legal_code_2.drop_duplicates(subset='loc_name', keep='last').dropna().reset_index(drop=True)
for i in range(len(legal_code_2)):
    legal_code_2.loc_name[i] = romanise(legal_code_2.loc_name[i], 'rr').replace('-', '').title()
legal_code_2.loc_name = legal_code_2.loc_name.str.replace('si', '')
legal_code_2.loc_name = legal_code_2.loc_name.str.replace('gun', '')
legal_code_2.loc_name = legal_code_2.loc_name.str.replace('gu', '')
legal_code_2

,PNU,loc_name
0,1111,Jongno
1,1117,Yongsan
2,1120,Seongdong
3,1121,Gwangjin
4,1123,Dongdaemun
...,...,...
239,4887,Hamyang
240,4888,Geochang
241,4889,Hapcheon
242,5011,Jeju


## (3) 지역 이름 출력

In [8]:
legal_code = pd.concat([legal_code_1, legal_code_2]).reset_index(drop=True)
legal_code['loc_name'].unique()

array(['Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon',
       'Ulsan', 'Sejongteukbyeoljachisi', 'Bukbuchuljangso', 'Gyeonggido',
       'Donghaechuljangso', 'Gangwondo', 'Chungcheongbukdo',
       'Chungcheongnamdo', 'Jeollabukdo', 'Jeollanamdo',
       'Gyeongsangbukdo', 'Gyeongsangnamdo', 'Jeju', 'Jongno', 'Yongsan',
       'Seongdong', 'Gwangjin', 'Dongdaemun', 'Jungnang', 'Seongbuk',
       'Gangbuk', 'Dobong', 'Nowon', 'Eunpyeong', 'Seodaemun', 'Mapo',
       'Yangcheon', 'Guro', 'Geumcheon', 'Yeongdeungpo', 'Dongjak',
       'Gwanak', 'Seocho', 'Gangnam', 'Songpa', 'Gangdong', 'Yeongdo',
       'Busanjin', 'Dongnae', 'Haeundae', 'Saha', 'Geumjeong', 'Gangseo',
       'Yeonje', 'Suyeong', 'Sasang', 'Gijang', 'Suseong', 'Dalseo',
       'Dalseong', 'Jungyeongjongchuljang', 'Jungyongyuchuljang',
       'Michuhol', 'Yeonsu', 'Namdong', 'Bupyeong', 'Gyeyang',
       'Seogeomdanchuljang', 'Ganghwa', 'Ongjin', 'Gwangsan', 'Seo',
       'Yuseong', 'Daedeok', 'Jung', 'Nam', '

# 3. 법정동코드와 공시지가 병합

In [9]:
economy_d = pd.merge(economy_df, legal_code, on="PNU", how="inner")

In [10]:
economy_d

,PNU,price,loc_name
0,1111,5789.0,Jongno
1,1117,6553.0,Yongsan
2,1120,4353.0,Seongdong
3,1121,4157.0,Gwangjin
4,1123,3587.0,Dongdaemun
...,...,...,...
256,4889,20.0,Gyeongsangnamdo
257,4889,20.0,Hapcheon
258,5011,223.0,Jeju
259,5013,137.0,Jeju


In [11]:
with open('../pkl/economy_df.pickle','wb') as fw:
    pickle.dump(economy_d, fw)